# Rozwiązywanie układów równań liniowych metodami interacyjnymi
## Zadanie 1
Zaimplementuj metodę Jacobiego. Podaj warunki jej stosowalności. Wygeneruj co najmniej trzy odpowiednie macierze o różnych wielkościach i sprawdź działanie swojej metody. Zwróć uwagę na zbieżność tej metody.

Warunki na stosowanie metody Jacobiego:  
- Macierz współczynników jest nieredukowalna
- Diagonalna jest dominująca


In [1]:
import numpy as np
import random


def jacobi(A, b, eps=1e-10, n=500):
    D = np.diag(np.diag(A))
    LU = A - D
    x = np.zeros(len(b))
    it = 0

    for i in range(n):
        D_inv = np.diag(1 / np.diag(D))
        x_new = np.dot(D_inv, b - np.dot(LU, x))
        it += 1
        if np.linalg.norm(x_new - x) < eps:
            return x_new, it
        x = x_new

    return x, it


def generate_random_matrix_vector(m_min, m_max, v_min, v_max, n):
    A = np.zeros((n, n))
    b = np.random.uniform(low=v_min, high=v_max + 1, size=(n,))

    for i in range(n):
        sum = 0

        for j in range(n):
            if j >= i:
                A[i][j] = random.randint(m_min, m_max)
                A[j][i] = A[i][j]
            if i != j:
                sum += abs(A[i][j])

        A[i][i] = sum

    return A, b

def comparison(m_min, m_max, v_min, v_max, n_size, n_iter, eps=1e-10):
    A, b = generate_random_matrix_vector(m_min, m_max, v_min, v_max, n_size)
    print(f"Numpy\n{np.linalg.solve(A, b)}")
    print(f"Jacobi method:\n{jacobi(A, b, eps, n_iter)}\n\n")
    

In [2]:
comparison(-15, 15, -10, 10, 5, 1000)

Numpy
[-0.57912578  0.32403963 -0.61532504 -0.08008715  0.07809279]
Jacobi method:
(array([-0.57912578,  0.32403963, -0.61532504, -0.08008715,  0.07809279]), 107)




In [3]:
comparison(-25, 25, -5, 5, 5, 1000)

Numpy
[-0.22368406 -0.27305008  0.20425408 -0.23508445 -0.13353158]
Jacobi method:
(array([-0.22368406, -0.27305008,  0.20425408, -0.23508445, -0.13353158]), 145)




In [4]:
comparison(-15, 15, -20, 20, 5, 1000)

Numpy
[ 0.50933203  0.44974506  0.32206853 -0.26102059  0.49737241]
Jacobi method:
(array([ 0.50933203,  0.44974506,  0.32206853, -0.26102059,  0.49737241]), 95)




## Zadanie 2
Zaimplementuj metodę Gaussa-Seidla i kolejnych nadrelaksacji (successive over-relaxation). 
Podaj warunki stosowalności tych metod. Przeprowadź badanie działania swoich implementacji 
analogicznie jak w poprzednim zadaniu. Porównaj zbieżność wszystkich trzech metod.

Warunki:
- Gaussa-Seidela
    - Warunki konieczne dla metody Jacobiego
    - Dla macierzy określonej dodatnio jest zbieżna dla dowolnego wektora początkowego 
- SOR  
    - Warunki Gaussa-Seidla 
    - Przypsiesza ona zbieżność ciągu przez przemnożenie przez odpowiednio dobraną liczbę $\omega$.  
     Parametr $\omega$ służy do polepszania (przyspieszania) zbiezności metody i może przyjmować wartości z przedziału (0, 2). Dla pozostałych wartości metoda może nie być zbieżna dla pewnych przybliżeń początkowych.

In [5]:
def gauss_seidel(A, b, n, eps):
    x = np.zeros_like(b)
    it = 0
    for _ in range(n):
        x_new = np.zeros_like(x)
        it += 1
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.allclose(x, x_new, atol=eps, rtol=0):
            return x_new, it
        x = x_new
    return x, it


def sor(A, b, n, eps, omg):
    if omg < 0 or omg > 2:
        raise Exception("Omega must be from range (0, 2)")
    it = 0
    x = np.zeros_like(b)
    x_new = np.zeros_like(x)
    for _ in range(n):
        it += 1
        for i in range(b.shape[0]):
            old_sum = np.dot(A[i, i + 1:], x_new[i + 1:])
            new_sum = np.dot(A[i, :i], x[:i])
            x[i] = (b[i] - (old_sum + new_sum)) / A[i, i]
            x[i] = np.dot(x[i], omg) + np.dot(x_new[i], (1 - omg))
        if np.linalg.norm(np.dot(A, x) - b) < eps:
            return x_new, it
        x_new = x
    return x, it


def comparison2(m_min, m_max, v_min, v_max, n_size, n_iter, eps=1e-10, omg=1.5):
    A, b = generate_random_matrix_vector(m_min, m_max, v_min, v_max, n_size)
    print(f"Numpy:\n{np.linalg.solve(A, b)}")
    print(f"Gauss_Seidel:\n{gauss_seidel(A, b, n_iter, eps)[0]}")
    print(f"SOR:\n{sor(A, b, n_iter, eps, omg)[0]}")
    

In [6]:
comparison2(-10, 10, -10, 10, 10, 1000)

Numpy:
[ 0.17706927  0.08494941 -0.01012123  0.325486    0.34270601  0.28170022
  0.03271042 -0.37460411  0.33149447  0.23502738]
Gauss_Seidel:
[ 0.17706927  0.08494941 -0.01012123  0.325486    0.34270601  0.28170022
  0.03271042 -0.37460411  0.33149447  0.23502738]
SOR:
[ 0.17706927  0.08494941 -0.01012123  0.325486    0.34270601  0.28170022
  0.03271042 -0.37460411  0.33149447  0.23502738]


In [7]:
comparison2(-20, 20, -20, 20, 20, 1000)

Numpy:
[-0.0719628  -0.09946151  0.09150158  0.08018565 -0.07412578 -0.03346887
 -0.0144381   0.02767503  0.07741711 -0.01195307  0.06296564 -0.08489765
  0.08818994  0.07489812  0.1278588   0.05194139  0.10705735 -0.07320339
  0.00781496  0.04799348]
Gauss_Seidel:
[-0.0719628  -0.09946151  0.09150158  0.08018565 -0.07412578 -0.03346887
 -0.0144381   0.02767503  0.07741711 -0.01195307  0.06296564 -0.08489765
  0.08818994  0.07489812  0.1278588   0.05194139  0.10705735 -0.07320339
  0.00781496  0.04799348]
SOR:
[-0.0719628  -0.09946151  0.09150158  0.08018565 -0.07412578 -0.03346887
 -0.0144381   0.02767503  0.07741711 -0.01195307  0.06296564 -0.08489765
  0.08818994  0.07489812  0.1278588   0.05194139  0.10705735 -0.07320339
  0.00781496  0.04799348]


In [8]:
comparison2(-100, 100, -100, 100, 30, 1000)


Numpy:
[-0.02018734  0.07815528  0.01572999  0.05910823 -0.01314568 -0.02234894
 -0.01757521  0.07178803  0.03367039  0.02627887 -0.02445343 -0.01917131
 -0.00066329  0.01887237  0.03073744 -0.05697529 -0.04273039 -0.03528191
 -0.01889978 -0.05330959 -0.07245089 -0.01586593 -0.02645346  0.08413302
 -0.03186904 -0.02558981 -0.08765459  0.05729231  0.03441128 -0.0607838 ]
Gauss_Seidel:
[-0.02018734  0.07815528  0.01572999  0.05910823 -0.01314568 -0.02234894
 -0.01757521  0.07178803  0.03367039  0.02627887 -0.02445343 -0.01917131
 -0.00066329  0.01887237  0.03073744 -0.05697529 -0.04273039 -0.03528191
 -0.01889978 -0.05330959 -0.07245089 -0.01586593 -0.02645346  0.08413302
 -0.03186904 -0.02558981 -0.08765459  0.05729231  0.03441128 -0.0607838 ]
SOR:
[-0.02018734  0.07815528  0.01572999  0.05910823 -0.01314568 -0.02234894
 -0.01757521  0.07178803  0.03367039  0.02627887 -0.02445343 -0.01917131
 -0.00066329  0.01887237  0.03073744 -0.05697529 -0.04273039 -0.03528191
 -0.01889978 -0.0533095

In [9]:
def comparison3(m_min, m_max, v_min, v_max, n_size, n_iter, eps=1e-10, omg=1.5):
    A, b = generate_random_matrix_vector(m_min, m_max, v_min, v_max, n_size)
    x, it = jacobi(A, b, eps, n_iter)
    print(f"Jacobi:\n Iterations: {it}\n{x}")
    x, it = gauss_seidel(A, b, n_iter, eps)
    print(f"Gauss_Seidel:\nIterations: {it}\n{x}")
    x, it = sor(A, b, n_iter, eps, omg)
    print(f"SOR:\nIterations: {it}\n{x}")
    

In [10]:
comparison3(-10, 10, -10, 10, 10, 1000)

Jacobi:
 Iterations: 40
[-0.0715616   0.06813206 -0.16262818  0.11793703 -0.10069304 -0.1574273
  0.03091571 -0.11368387 -0.02037658 -0.17445372]
Gauss_Seidel:
Iterations: 17
[-0.0715616   0.06813206 -0.16262818  0.11793703 -0.10069304 -0.1574273
  0.03091571 -0.11368387 -0.02037658 -0.17445372]
SOR:
Iterations: 22
[-0.0715616   0.06813206 -0.16262818  0.11793703 -0.10069304 -0.1574273
  0.03091571 -0.11368387 -0.02037658 -0.17445372]


In [11]:
comparison3(-20, 20, -20, 20, 20, 1000)

Jacobi:
 Iterations: 28
[ 0.04145785 -0.03520526  0.02524139 -0.00795463  0.0694337  -0.06251523
 -0.03074765  0.03179543  0.00921414 -0.05331599 -0.03481308  0.02569037
  0.06044125 -0.08826812  0.02780301 -0.06048493 -0.06654238 -0.06541194
  0.07262225 -0.08104857]
Gauss_Seidel:
Iterations: 16
[ 0.04145785 -0.03520526  0.02524139 -0.00795463  0.0694337  -0.06251523
 -0.03074765  0.03179543  0.00921414 -0.05331599 -0.03481308  0.02569037
  0.06044125 -0.08826812  0.02780301 -0.06048493 -0.06654238 -0.06541194
  0.07262225 -0.08104857]
SOR:
Iterations: 19
[ 0.04145785 -0.03520526  0.02524139 -0.00795463  0.0694337  -0.06251523
 -0.03074765  0.03179543  0.00921414 -0.05331599 -0.03481308  0.02569037
  0.06044125 -0.08826812  0.02780301 -0.06048493 -0.06654238 -0.06541194
  0.07262225 -0.08104857]


In [12]:
comparison3(-100, 100, -100, 100, 30, 1000)

Jacobi:
 Iterations: 20
[-0.02632463  0.02545171  0.01063524  0.00721894  0.02824821  0.07633452
  0.01366397  0.01519488 -0.02461543 -0.04929148 -0.06796769  0.04269941
 -0.01299028 -0.06582703 -0.03151276 -0.04586668 -0.00391605  0.02970699
 -0.06402862  0.04212487 -0.06893208 -0.05842069 -0.05936808 -0.03729884
  0.06856572 -0.03605369 -0.00969683  0.02995482 -0.01111076  0.0267059 ]
Gauss_Seidel:
Iterations: 12
[-0.02632463  0.02545171  0.01063524  0.00721894  0.02824821  0.07633452
  0.01366397  0.01519488 -0.02461543 -0.04929148 -0.06796769  0.04269941
 -0.01299028 -0.06582703 -0.03151276 -0.04586668 -0.00391605  0.02970699
 -0.06402862  0.04212487 -0.06893208 -0.05842069 -0.05936808 -0.03729884
  0.06856572 -0.03605369 -0.00969683  0.02995482 -0.01111076  0.0267059 ]
SOR:
Iterations: 15
[-0.02632463  0.02545171  0.01063524  0.00721894  0.02824821  0.07633452
  0.01366397  0.01519488 -0.02461543 -0.04929148 -0.06796769  0.04269941
 -0.01299028 -0.06582703 -0.03151276 -0.04586668 